In [ ]:
## setup analysis
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
from analysis.lib.purification import purify_analysis as pa; reload(pa)
%matplotlib inline

theta = 'pi/4'


In [ ]:
folder_lt3,folder_lt4 = pa.get_folders_from_theta(theta)
ssro_calib_lt3,ssro_calib_lt4 = pa.get_ssro_calibs()

Purify_XX = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
Purify_YY = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
Purify_ZZ = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)

Purify_XX.get_tstamps_and_offsets(contains = 'XX')
Purify_YY.get_tstamps_and_offsets(contains = 'YY')
Purify_ZZ.get_tstamps_and_offsets(contains = 'ZZ')


In [ ]:
### load the data
Purify_XX.load_raw_data()
Purify_XX.correct_pq_times()
Purify_YY.load_raw_data()
Purify_YY.correct_pq_times()
Purify_ZZ.load_raw_data()
Purify_ZZ.correct_pq_times()

In [ ]:
def sweep_parameter(param_name,sweep_pts,apply_ROC, plot_all = False):
    
    ## get correlation probabilties
    psi_m_XX,psi_p_XX,psi_m_XX_corrs,psi_p_XX_corrs = Purify_XX.sweep_filter_parameter_vs_correlations(param_name,sweep_pts,
                                                                         apply_ROC = apply_ROC,do_plot= False)
    psi_m_YY,psi_p_YY,psi_m_YY_corrs,psi_p_YY_corrs = Purify_YY.sweep_filter_parameter_vs_correlations(param_name,sweep_pts,
                                                                         apply_ROC = apply_ROC,do_plot= False)
    psi_m_ZZ,psi_p_ZZ,psi_m_ZZ_corrs,psi_p_ZZ_corrs = Purify_ZZ.sweep_filter_parameter_vs_correlations(param_name,sweep_pts,
                                                                         apply_ROC = apply_ROC,do_plot= False)       
    ## calculate fidelities
    psi_m_F = (psi_m_XX[1] + psi_m_YY[1] + psi_m_ZZ[1]+1)/4.
    psi_p_F = (psi_p_XX[1] + psi_p_YY[1] + psi_p_ZZ[1]+1)/4.
      
    ## calciulate error bars
    psi_m_F_u = np.sqrt(psi_m_XX[2]**2 + psi_m_YY[2]**2 + psi_m_ZZ[2]**2)/4.
    psi_p_F_u = np.sqrt(psi_p_XX[2]**2 + psi_p_YY[2]**2 + psi_p_ZZ[2]**2)/4.
    
    # calculate ebits    
#     ebits_p, ebits_p_u = pa.calculate_ebits(psi_m_YY,psi_m_ZZ,psi_m_XX_corrs)
#     ebits_m, ebits_m_u = pa.calculate_ebits(psi_p_YY,psi_p_ZZ,psi_p_XX_corrs)
    
    psi_F = 0.5*(psi_m_F+psi_p_F)
    psi_F_u = 0.5*np.sqrt(psi_m_F_u**2+psi_p_F_u**2)
    
    print psi_m_F[0],psi_m_F_u[0]
    print psi_p_F[0],psi_p_F_u[0]
    fig  = plt.figure()
    ax = plt.subplot(111)
    ax.set_xlabel(param_name)
    ax.set_ylabel('Fidelity')
    plt.errorbar(sweep_pts,psi_m_F,psi_m_F_u,label = '-')
    plt.errorbar(sweep_pts,psi_p_F,psi_p_F_u,label = '+')
    if param_name == 'bin_w2':
        plt.xlim(np.array(plt.xlim()) + [-50,50])
    plt.legend()
    plt.show()
    plt.close('all')
 
    fig  = plt.figure()
    ax = plt.subplot(111)
    ax.set_xlabel(param_name)
    ax.set_ylabel('Fidelity (avg)')
    plt.errorbar(sweep_pts,psi_F,psi_F_u)
    if param_name == 'bin_w2':
        plt.xlim(np.array(plt.xlim()) + [-50,50])
    plt.show()
    plt.close('all')
    
    fig  = plt.figure()
    ax = plt.subplot(111)
    ax.set_xlabel(param_name)
    ax.set_ylabel('ebits')

    plt.errorbar(sweep_pts,ebits_p,ebits_p_u,label = '+')
    plt.errorbar(sweep_pts,ebits_m,ebits_m_u,label = '-')
    if param_name == 'bin_w2':
        plt.xlim(np.array(plt.xlim()) + [-50,50])
    plt.legend()
    plt.show()
    plt.close('all')
    
    ### get the rate for any given data point
    ### keep in mind that this includes ALL down time of the experiment. Some might argue that this is an unfair comparison
    ### we therefore also include a diagram that uses the operation time of the sequence only and the number of times the sequence has run
    ### tomography is excluded from this calculated rate
    total_time = 0
    total_time += Purify_XX.get_total_time() + Purify_YY.get_total_time() + Purify_ZZ.get_total_time()
    total_counts = psi_m_XX[0]+psi_p_XX[0]+psi_m_YY[0]+psi_p_YY[0]+psi_m_ZZ[0]+psi_p_ZZ[0]
    fig  = plt.figure()
    ax = plt.subplot(111)
    ax.set_xlabel(param_name)
    ax.set_ylabel('Rate (Hz)')
    plt.plot(sweep_pts,total_counts/total_time, label = 'incl. overhead')
    if param_name == 'bin_w2':
        plt.xlim(np.array(plt.xlim()) + [-50,50]) 
    total_time = 0
    total_time += Purify_XX.estimate_sequence_time() + Purify_YY.estimate_sequence_time() + Purify_ZZ.estimate_sequence_time()
    
    plt.plot(sweep_pts,total_counts/total_time, label = 'excl. overhead')
    if param_name == 'bin_w2':
        plt.xlim(np.array(plt.xlim()) + [-50,50])
    plt.legend(loc = 2)
    plt.show()
    plt.close('all')
    
    ## plot individual correlations
    if plot_all:
        fig  = plt.figure()
        ax = plt.subplot(111)
        ax.set_xlabel(param_name)
        ax.set_ylabel('Expectation value')
        plt.errorbar(sweep_pts,psi_m_XX[1],psi_m_XX[2],label = 'XX')
        plt.errorbar(sweep_pts,psi_m_YY[1],psi_m_YY[2],label = 'YY')
        plt.errorbar(sweep_pts,psi_m_ZZ[1],psi_m_ZZ[2],label = 'ZZ')
        if param_name == 'bin_w2':
            plt.xlim(np.array(plt.xlim()) + [-50,50])   
        plt.title('Psi_minus correlations')
        plt.legend()
        plt.show()
        plt.close('all')
        
        fig  = plt.figure()
        ax = plt.subplot(111)
        ax.set_xlabel(param_name)
        ax.set_ylabel('Expectation value')
        plt.errorbar(sweep_pts,psi_p_XX[1],psi_p_XX[2],label = 'XX')
        plt.errorbar(sweep_pts,psi_p_YY[1],psi_p_YY[2],label = 'YY')
        plt.errorbar(sweep_pts,psi_p_ZZ[1],psi_p_ZZ[2],label = 'ZZ')
        if param_name == 'bin_w2':
            plt.xlim(np.array(plt.xlim()) + [-50,50])
        plt.title('Psi_plus correlations')
        plt.legend()
        plt.show()
        plt.close('all')

In [ ]:
sweep_parameter('bin_w2',np.arange(50,450,50),apply_ROC = True,plot_all = False)

In [ ]:
sweep_parameter('st_start',np.linspace(-4e3,4e3,10)+2773.5e3,apply_ROC = True)

In [ ]:
#### some plotting globals
lw = 1.
fontsize = 11.
markersize = 3
color_list = ['#4FA6FF','#50A53A','orange','Gray','yellow']

#### compare correlations in one plot + nicer plotting
#### also gets the fidelity etc.
#### we only look at average correlations (so averaged over the plus and minus signature).
def load_data(theta):
    folder_lt3,folder_lt4 = pa.get_folders_from_theta(theta)
    ssro_calib_lt3,ssro_calib_lt4 = pa.get_ssro_calibs()

    Purify_XX = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
    Purify_YY = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
    Purify_ZZ = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)

    Purify_XX.get_tstamps_and_offsets(contains = 'XX')
    Purify_YY.get_tstamps_and_offsets(contains = 'YY')
    Purify_ZZ.get_tstamps_and_offsets(contains = 'ZZ')
    
    Purify_XX.load_raw_data()
    Purify_XX.correct_pq_times()
    Purify_YY.load_raw_data()
    Purify_YY.correct_pq_times()
    Purify_ZZ.load_raw_data()
    Purify_ZZ.correct_pq_times()
    
    return Purify_XX,Purify_YY,Purify_ZZ
    
def get_corrs(param_name,sweep_pts,Purify_XX,Purify_YY,Purify_ZZ,apply_ROC = True):
    ## get correlation probabilties
    psi_m_XX,psi_p_XX,psi_m_XX_corrs,psi_p_XX_corrs = Purify_XX.sweep_filter_parameter_vs_correlations(param_name,sweep_pts,
                                                                         apply_ROC = apply_ROC,do_plot= False)
    psi_m_YY,psi_p_YY,psi_m_YY_corrs,psi_p_YY_corrs = Purify_YY.sweep_filter_parameter_vs_correlations(param_name,sweep_pts,
                                                                         apply_ROC = apply_ROC,do_plot= False)
    psi_m_ZZ,psi_p_ZZ,psi_m_ZZ_corrs,psi_p_ZZ_corrs = Purify_ZZ.sweep_filter_parameter_vs_correlations(param_name,sweep_pts,
                                                                         apply_ROC = apply_ROC,do_plot= False)       
    ## calculate fidelities
    psi_m_F = (psi_m_XX[1] + psi_m_YY[1] + psi_m_ZZ[1]+1)/4.
    psi_p_F = (psi_p_XX[1] + psi_p_YY[1] + psi_p_ZZ[1]+1)/4.
      
    ## calciulate error bars
    psi_m_F_u = np.sqrt(psi_m_XX[2]**2 + psi_m_YY[2]**2 + psi_m_ZZ[2]**2)/4.
    psi_p_F_u = np.sqrt(psi_p_XX[2]**2 + psi_p_YY[2]**2 + psi_p_ZZ[2]**2)/4.
    
    # calculate ebits    
    #ebits_p, ebits_p_u = pa.calculate_ebits(psi_m_YY,psi_m_ZZ,psi_m_XX_corrs)
    #ebits_m, ebits_m_u = pa.calculate_ebits(psi_p_YY,psi_p_ZZ,psi_p_XX_corrs)
    
    psi_F = 0.5*(psi_m_F+psi_p_F)
    psi_F_u = 0.5*np.sqrt(psi_m_F_u**2+psi_p_F_u**2)
    
    psi_XX    = (psi_m_XX[1]+psi_p_XX[1])/2.
    psi_YY    = (psi_m_YY[1]+psi_p_YY[1])/2.
    psi_ZZ    = (psi_m_ZZ[1]+psi_p_ZZ[1])/2.
    psi_XX_u  = np.sqrt(psi_m_XX[2]**2+psi_p_XX[2]**2)/2.
    psi_YY_u  = np.sqrt(psi_m_YY[2]**2+psi_p_YY[2]**2)/2.
    psi_ZZ_u  = np.sqrt(psi_m_ZZ[2]**2+psi_p_ZZ[2]**2)/2.
    
    return psi_F,psi_F_u,psi_XX,psi_XX_u,psi_YY,psi_YY_u,psi_ZZ,psi_ZZ_u

def plot_values_bars(x_list,y_list,y_u_list):
    
    for x,y,y_u,color in zip(x_list,y_list,y_u_list,color_list):
        plt.errorbar(x-25,y,y_u,fmt = 'o',color = color,ms=markersize,mew=lw/2.,zorder=3) #formatting still needs to be done
        ### we want to include bars to signify integration here
        barwidth = x[1]-x[0]

        ### different alphas for the different bars. depending on the absolute height.
        if np.amax(y)>0.5:
            alpha = 0.07
        else:
            alpha = 0.12
        plt.bar(x-barwidth,y,width = barwidth,alpha = alpha,color = color,zorder=0)
        
def plot_model_fidelity(theta,ax,col):
    
    if theta == 'pi/8':
        model_x =  np.arange(50,450,50)
        model_y = np.array([0.692477, 0.635345, 0.562946, 0.49924, 0.456242, 0.431382, 0.417291, 0.40859])
    elif theta == 'pi/6':
        model_x =  np.arange(50,450,50)
        model_y = np.array([0.655212, 0.591695, 0.513582, 0.448808, 0.407071, 0.383613, 0.370692, 0.363049])
    elif theta == 'pi/5':
        model_x =  np.arange(50,450,50)
        model_y = np.array([0.601551, 0.532121, 0.450335, 0.386448, 0.346552, 0.324302, 0.312279, 0.305567])
    elif theta == 'pi/4':
        model_x =  np.arange(50,450,50)
        model_y = np.array([0.554004, 0.482161, 0.400364, 0.338637, 0.300365, 0.278844, 0.267288, 0.261111])
        
    ax.plot(model_x-25,model_y,'--',color=col,lw=lw,zorder=1)
    
def plot_model_correlations(theta,corrs,col,ax):

    model_x = np.arange(50,450,50)
    if theta == 'pi/8':
        if corrs == 'XX':
            model_y = np.array([0.484095, 0.395607, 0.288232, 0.19235, 0.124238, 0.0819756, 0.0564024, 0.0399623])
        elif corrs == 'YY':
            model_y = np.array([0.487313, 0.399866, 0.293623, 0.198022, 0.129337, 0.08614, 0.0596311, 0.042385])
        elif corrs == 'ZZ':
            model_y = np.array([0.798499, 0.745908, 0.669928, 0.606588, 0.571395, 0.557412, 0.553129, 0.552014])
        else:
            plot_model_fidelity(theta,ax,col)
            return
    elif theta == 'pi/6':
        if corrs == 'XX':
            model_y = np.array([0.444586, 0.352662, 0.245903, 0.157866, 0.0995197, 0.064889, 0.0444218, 0.0314118])
        elif corrs == 'YY':
            model_y = np.array([0.44754, 0.356451, 0.250493, 0.162516, 0.103603, 0.0681851, 0.0469646, 0.0333161])
        elif corrs == 'ZZ':
            model_y = np.array([0.728724, 0.657665, 0.557931, 0.474851, 0.425161, 0.401378, 0.391383, 0.387467])
        else:
            plot_model_fidelity(theta,ax,col)
            return
    elif theta == 'pi/5':
        if corrs == 'XX':
            model_y = np.array([0.38812, 0.294993, 0.193562, 0.117875, 0.0718017, 0.0459578, 0.0311856, 0.0219682])
        elif corrs == 'YY':
            model_y = np.array([0.390697, 0.298154, 0.197166, 0.121342, 0.074746, 0.0482918, 0.0329707, 0.0233])
        elif corrs == 'ZZ':
            model_y = np.array([0.627386, 0.535338, 0.410611, 0.306574, 0.239661, 0.202957, 0.18496,0.176998])
        else:
            plot_model_fidelity(theta,ax,col)
            return
    elif theta == 'pi/4':
        if corrs == 'XX':
            model_y = np.array([0.338625, 0.247738, 0.154232, 0.0897757, 0.053049, 0.0333666, 0.0224411, 0.0157448])
        elif corrs == 'YY':
            model_y = np.array([0.340872, 0.250386, 0.157097, 0.0924132, 0.0552232, 0.0350609, 0.0237256, 0.0166993])
        elif corrs == 'ZZ':
            model_y = np.array([0.536517, 0.430521, 0.290125, 0.172357, 0.0931864, 0.0469466, 0.0229842, 0.0119997])
        else:
            plot_model_fidelity(theta,ax,col)
            return
    ax.plot(model_x-25,model_y,'--',color=col,lw=lw,zorder=1)


In [ ]:
####
theta = 'pi/8' #'pi/8' or pi/6 etc.
XX,YY,ZZ = load_data(theta)

In [ ]:
x_sweep = np.arange(50,450,50)
psi_F,psi_F_u,psi_XX,psi_XX_u,psi_YY,psi_YY_u,psi_ZZ,psi_ZZ_u = get_corrs('bin_w2',x_sweep,XX,YY,ZZ,apply_ROC = True)

In [ ]:
y_list = [psi_F,psi_XX,psi_YY,psi_ZZ];
x_list = [x_sweep]*len(y_list);
y_u_list = [psi_F_u,psi_XX_u,psi_YY_u,psi_ZZ_u];

In [ ]:
#### make a figure with 4 subplots that shows each decay individually, including the model.
save_folder = r'K:\ns\qt\Diamond\Projects\Purification\Paper\Plots'
golden_ratio = 1.61803
mm_to_inches = 0.0393701 ### inch/mm
figure_width = mm_to_inches*83
fig,((ax1,ax2),(ax3,ax4))=plt.subplots(2, 2, sharex='col', sharey='row',figsize =(figure_width,figure_width/golden_ratio))


color_list = ['#3594F2','#50A53A','#F29335','Gray']
lw = 1.
fontsize = 10.
markersize = 3
fig.text(0.5, -0.04, 'Entangling attempts (binned)', ha='center', va='center')


legend_list = ['Fidelity',
               r'$| \langle \hat{X}\hat{X} \rangle |$',
               r'$| \langle \hat{Y}\hat{Y} \rangle |$',
               r'$| \langle \hat{Z}\hat{Z} \rangle |$']
corrs_list = ['Fidelity','ZZ','YY','XX']
for ax,x,y,y_u,c,l,corrs in zip([ax1,ax2,ax3,ax4],x_list,y_list,y_u_list,color_list,legend_list,corrs_list):
    ax.tick_params(labelsize = fontsize,width =lw)
    ax.set_xticks([25,125,225,325])
    ax.set_xticks([75,175,275,375],minor= True)
    ax.set_yticks([0,0.5,1])
    ax.set_yticks([0.25,0.75],minor= True)
    ax.set_ylim([0,1])
    ax.set_xlim([0,400])
    if theta == 'pi/4':
        ax.set_xlim([0,250])
    ax.set_ylabel(l)
    if (ax == ax2) or (ax == ax4):
        ax.yaxis.set_label_position('right')
    ax.xaxis.set_tick_params(which = 'both',width=lw)
    ax.yaxis.set_tick_params(which = 'both',width=lw)
    [i.set_linewidth(lw) for i in ax.spines.itervalues()]
    ax.errorbar(x-25,y,y_u,fmt = 'o',color = c,ms=markersize,mew=lw/2.,zorder=3) 
    ### we want to include bars to signify integration here
    barwidth = (x[1]-x[0])/1.5
    ax.bar(x-25-barwidth/2,y,width = barwidth,alpha = 0.2,color = c,zorder=0,linewidth=0.5)
    

    plot_model_correlations(theta,corrs,c,ax)


plt.subplots_adjust(wspace=0.04)
# plt.savefig(os.path.join(save_folder,'binned_corrs_foursubplots_'+theta[-1]+'.png'),format='png',bbox_inches = 'tight',pad_inches=0.3)
# plt.savefig(os.path.join(save_folder,'binned_corrs_foursubplots_'+theta[-1]+'.pdf'),format='pdf',bbox_inches = 'tight',pad_inches=0.3)
plt.show()

In [ ]:
i = 3
# print x_sweep
print np.round(y_list[i],2)
print np.round(y_u_list[i],2)
